In [ ]:
import numpy as np
import plotly.graph_objects as go
from sklearn.datasets import make_classification

## Generamos nuestra base de datos

In [ ]:
X, y = make_classification(n_features=2, 
                           n_classes=2, 
                           n_informative=2, 
                           n_redundant=0, 
                           n_clusters_per_class=1)
Y = y.reshape(-1,1)

Graficamos nuestros datos

In [ ]:
mk = ['circle','cross']
graphs = []
for i in range(2):
  idx = Y[:,0]==i
  graphs.append(go.Scatter(mode='markers',\
                           x=X[idx,0],\
                           y=X[idx,1],\
                           name='Class %d'%(i),
                           marker=dict(symbol=mk[i],size=15)))
fig = go.Figure(data=graphs)
fig.update_layout(xaxis_title='$X_1$',yaxis_title='$X_2$')

Analizando los datos de la gráfica, obtenemos un plano separador de la forma $$X_{2}=mX_{1} + b$$
Según los datos de esta gráfica, estimaremos $b=0.1$ y $m=-1$.

Creando los datos de estimación del plano separador y superponiendo a la gráfica de los datos tenemos.


In [ ]:
m = -1.0
b = 0.0

# Generamos datos de la linea del plano separador.
xt = np.linspace(X[:,0].min(), X[:,0].max(), 100)
yt = m*xt + b

# Generamos la grilla de datos para observar el plano separador segun la regresion lineal
XX1, XX2 = np.meshgrid(np.linspace(X[:,0].min(), X[:,0].max(), 50),
                       np.linspace(X[:,1].min(), X[:,1].max(), 50))
Z = b + m*XX1 - XX2


from plotly.subplots import make_subplots


fig = make_subplots(rows=1, cols=2 ,specs=[[{'type':'contour'}, {'type': 'scene'}]],subplot_titles=['Mapa de contorno','Mapa de Superficie'])

fig.add_trace(go.Contour(x=np.linspace(X[:,0].min(), X[:,0].max(), 50),
                         y=np.linspace(X[:,1].min(), X[:,1].max(), 50),
                         z=Z,colorbar_x=0.43, colorbar_thickness=23), row=1, col=1)

pp=go.Surface(x=np.linspace(X[:,0].min(), X[:,0].max(), 50),
                         y=np.linspace(X[:,1].min(), X[:,1].max(), 50),
                         z=Z, colorbar_x=1.0075, colorbar_thickness=23)
fig.add_trace(pp, row=1, col=2)


for i in range(2):
  idx = Y[:,0]==i
  fig.add_trace(go.Scatter3d(mode='markers',\
                           x=X[idx,0],\
                           y=X[idx,1],\
                           z=np.zeros(shape=X[idx,0].shape),
                           name='Class s%d'%(i),
                           marker=dict(symbol=mk[i])),row=1, col=2)
  fig.add_trace(go.Scatter(mode='markers',\
                           x=X[idx,0],\
                           y=X[idx,1],
                           name='Class %d'%(i),
                           marker=dict(symbol=mk[i],size=10)),row=1, col=1)

fig.add_trace(go.Scatter3d(mode='lines', x=xt, y=yt,z=np.zeros(shape=xt.shape),line=dict(color='rgb(150,150,150)', width=10)),row=1, col=2)
fig.add_trace(go.Scatter(mode='lines', x=xt, y=yt,line=dict(color='rgb(150,150,150)', width=10)),row=1, col=1)
fig.update_layout(showlegend=False)

fig.show()

## Modelo de regresión logística

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X, Y.reshape(-1))

print(model.intercept_, model.coef_)

[0.90724164] [[-2.3561287  -2.10246508]]


El modelo de regresión logística según los datos obtenidos es:

\begin{align}
\mathcal{J} & =\frac{1}{1+e^{-X\Theta}}\\
 & =\frac{1}{1+e^{- \left(\theta_{0}+\theta_{1}x_{1}+\theta_{2}x_{2}\right)}}\\
&= \frac{1}{1+e^{- \left(0.907-2.356x_{1}-2.102x_{2}\right)}}
\end{align}

Si consideramos $\mathcal{J}=0.5$ podremos graficar la línea del plano separador en la coordenada $z=0.5$. Despejando $x_2$ en términos de $x_1$ tenemos

\begin{align}
0.5 &= \frac{1}{1+e^{- \left(0.907-2.356x_{1}-2.102x_{2}\right)}}\\
1+e^{- \left(0.907-2.356x_{1}-2.102x_{2}\right)} & = \frac{1}{0.5}\\
e^{- \left(0.907-2.356x_{1}-2.102x_{2}\right)} & = \frac{1}{0.5}-1\\
- \left(0.907-2.356x_{1}-2.102x_{2}\right) & = \ln\left(\frac{1}{0.5}-1\right)\\
 2.102x_{2} & = \ln\left(\frac{1}{0.5}-1\right)+0.907-2.356x_{1}\\
 x_{2} & = \frac{\ln\left(\frac{1}{0.5}-1\right)+0.907-2.356x_{1}}{2.102}\\
 x_{2} & = \frac{0.907-2.356x_{1}}{2.102}
\end{align}


In [ ]:
xt = np.linspace(X[:,0].min(), X[:,0].max(), 100)
yt = (np.log(1/0.5-1) + model.intercept_[0] + model.coef_[0,0]*xt )/(-model.coef_[0,1])

In [ ]:
fig = go.Figure()
mk = ['circle','cross']

XXtest = np.c_[XX1.reshape(-1,1),XX2.reshape(-1,1)]
Z = model.predict(XXtest).reshape(XX1.shape)

fig.add_trace(go.Surface(x=np.linspace(X[:,0].min(), X[:,0].max(), 50),
                         y=np.linspace(X[:,1].min(), X[:,1].max(), 50),
                         z=Z))

for i in range(2):
  idx = Y[:,0]==i
  fig.add_trace(go.Scatter3d(mode='markers',\
                           x=X[idx,0],\
                           y=X[idx,1],\
                           z=0.5*np.ones(shape=X[idx,0].shape),
                           name='Class %d'%(i),
                           marker=dict(symbol=mk[i])))
  
fig.add_trace(go.Scatter3d(mode='lines', x=xt, y=yt,z=0.5*np.ones(shape=xt.shape),line=dict(color='rgb(150,150,150)', width=10)))
fig.update_layout(scene = dict(xaxis_title='X<sub>1</sub>',yaxis_title=r'X<sub>2</sub>',zaxis_title='Z'))
fig.update_layout(showlegend=False)
